In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PyPDF2
import textract
import pdfplumber
import base64
import re
import io
import sys
import os
import pdfid
import csv
from collections import Counter
from os import path
from glob import glob  
from itertools import chain
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import roc_curve
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [2]:
# Read a CSV File into a list 
csvRawData = []

StateDict = {}
StateInvDict = {}
StateCount = 0

DistrictDict = {}
DistrictInvDict = {}
DistrictCount = 0

CropYearDict = {}
CropYearInvDict = {}
CropYearCount = 0

CropSeasonDict = {}
CropSeasonInvDict = {}
SeasonCount = 0

CropTypeDict = {}
CropTypeInvDict = {}
CropTypeCount = 0

DataList  = []
LabelList = []

ignore_zero_rows = True

with open('crop_production.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    LabelList = []
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
        else:
            
            if ignore_zero_rows == True and row[6] == '':
                 continue
                    
            sub_input_row = []
            
            if row[0] not in StateDict :
                StateDict[row[0]] = StateCount
                StateCount = StateCount + 1
            sub_input_row.append(StateDict[row[0]])
            
            if row[1] not in DistrictDict :
                DistrictDict[row[1]] = DistrictCount
                DistrictCount = DistrictCount + 1
            sub_input_row.append(DistrictDict[row[1]])
            
            if row[2] not in CropYearDict :
                CropYearDict[row[2]] = CropYearCount
                CropYearCount = CropYearCount + 1
            sub_input_row.append(CropYearDict[row[2]])
            
            if row[3] not in CropSeasonDict :
                CropSeasonDict[row[3]] = SeasonCount
                SeasonCount = SeasonCount + 1      
            sub_input_row.append(CropSeasonDict[row[3]])
            
            if row[4] not in CropTypeDict :
                CropTypeDict[row[4]] = CropTypeCount
                CropTypeCount = CropTypeCount + 1      
            sub_input_row.append(CropTypeDict[row[4]])
            
            sub_input_row.append(float(row[5]))
            
            if row[6] == '' :
                LabelList.append(float(0))
            else : 
                LabelList.append(float(row[6]))
                
            DataList.append(sub_input_row)
        line_count += 1
        
    print(f'Processed {line_count} lines.')
    

cut = len(LabelList)
#cut = 100000

DataSet = np.array(DataList[:cut][:])
Labels = np.array(LabelList[:cut])

print(DataSet.shape)

#Get IVDicts for all the existing Dictionaries
StateInvDict = dict((v, k) for k, v in StateDict.items())
DistrictInvDict = dict((v, k) for k, v in DistrictDict.items())
CropYearInvDict = dict((v, k) for k, v in CropYearDict.items())
CropSeasonInvDict = dict((v, k) for k, v in CropSeasonDict.items())
CropTypeInvDict = dict((v, k) for k, v in CropTypeDict.items())

Column names are State_Name, District_Name, Crop_Year, Season, Crop, Area, Production
Processed 242362 lines.
(242361, 6)


In [3]:
def DictionaryIterator( Dict, List) :
    print(List)
    return 

DFRaw = pd.DataFrame({'State_Name': DataSet[:, 0], 'District_Name': DataSet[:, 1], 'Crop_Year':DataSet[:,2], 'Season':DataSet[:,3], 'Crop':DataSet[:,4], 'Area': DataSet[:,5], 'Production':Labels[:]}) 
DFRaw.head()

DF = pd.DataFrame({'State_Name': [StateInvDict[row] for row in DataSet[:, 0]] , 'District_Name': [DistrictInvDict[row] for row in DataSet[:, 1]], 'Crop_Year':[CropYearInvDict[row] for row in DataSet[:, 2]], 'Season':[CropSeasonInvDict[row] for row in DataSet[:, 3]], 'Crop':[CropTypeInvDict[row] for row in DataSet[:, 4]], 'Area': DataSet[:,5], 'Production':Labels[:]}) 
DF.head()



,State_Name,District_Name,Crop_Year,Season,Crop,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,720.0,165.0


In [4]:
DF = pd.DataFrame({'Crop_Type': list(set([CropTypeInvDict[row] for row in DataSet[:, 4]]))})
DF

,Crop_Type
0,Yam
1,Mango
2,Jute & mesta
3,Coriander
4,Snak Guard
...,...
119,Other Fresh Fruits
120,Varagu
121,Plums
122,Arecanut


In [5]:
def scaleData(scaler, dataSet):
    DataSet_Scaled = scaler.fit_transform(dataSet)
    return DataSet_Scaled

def model_train(model) :
        model_name = model.__class__.__name__
        fit=model.fit(X_trainScaled, y_trainScaled)
        y_pred=fit.predict(X_testScaled)
        r2=r2_score(y_testScaled, y_pred)
        model_train.append([model, model_name, r2])

In [6]:
#Read about MinMaxScaler and Standard Scaler
Scalers =[
    MinMaxScaler(),
    StandardScaler()
]

In [7]:
# Understand the working mechanism of different models
models = [
    #svm.SVR(),
    LinearRegression(),
    DecisionTreeRegressor(),
    GradientBoostingRegressor(n_estimators=200, random_state=0),
    RandomForestRegressor(n_estimators=200, random_state=0) ,
    #MLPRegressor(hidden_layer_sizes=(32,16,8),activation="tanh" ,random_state=1, max_iter=100)
]

In [8]:
#old_stdout = sys.stdout
#sys.stdout = open('OutputContents.txt', 'w')

for testSize in [0.33, 0.3, 0.25, 0.2] :
    
    print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;New Test Size;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
    print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
    print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
    print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
    print(";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;")
    
    print("Test Size Used is : ", testSize)
    for scale in Scalers:
        print("#######################################################################################################")
        print("#######################################################################################################")

        print("\n\nScaler Used is ", scale)
        scaleName = scale.__class__.__name__
        DataSetScaled = scaleData(scale, DataSet)
        LabelsScaled = Labels

        for useScaleForLabels in [0,1]:

            print("____________________________________________________________________________________________________")
            if(useScaleForLabels == 1):
                print("Using Scaling for Labels as well\n")
                LabelsScaled = scaleData(scale, Labels.reshape(-1,1))

            X_trainScaled, X_testScaled, y_trainScaled, y_testScaled = train_test_split(DataSetScaled, LabelsScaled, test_size = testSize)
            model_train = []

            for model in models :
                model_name = model.__class__.__name__
                fit=model.fit(X_trainScaled, y_trainScaled)
                y_pred=fit.predict(X_testScaled)

                r2=r2_score(y_testScaled, y_pred)
                model_train.append([model, model_name, r2])

            print("Models Trained With Scores are as follows:\n")
            print(*model_train, sep = "\n")

            # Now test all possible depths with decision tree regressor, Gradient Boosing Regressor, Random Forest Regressor
            print("\n\nDefault Decision Tree Depth with Scaler and setting ", scale, " is ", model_train[1][0].tree_.max_depth)
            print("\n\nDefault GradientBoostingRegressor Tree Depth with Scaler and setting ", scale, " is ", max(estimator.tree_.max_depth for estimator in chain.from_iterable(zip(*model_train[2][0].estimators_))))
            print("\n\nDefault RandomForestRegressor Tree Depth with Scaler and setting ", scale, " is ", max([estimator.get_depth() for estimator in model_train[3][0].estimators_]))
            
            '''
            TreeModelNames = ["", "", ""]
            max_Scores = [float(0), float(0), float(0)]
            max_ScoreDepths = [int(1), int(1), int(1)]
            
            for depth in range(50,100):
                
                TreeModels = [
                    DecisionTreeRegressor(max_depth= depth),
                    #GradientBoostingRegressor(n_estimators=200, max_depth = depth, random_state=0),
                    RandomForestRegressor(n_estimators=200, max_depth = depth, random_state=0) 
                ]
                
                counter  = int(0)
                for model in TreeModels:
                    
                    model_name = model.__class__.__name__
                    fit = model.fit(X_trainScaled, y_trainScaled)
                    y_pred=fit.predict(X_testScaled)
                    r2=r2_score(y_testScaled, y_pred)

                    if(r2 > max_Scores[counter]):
                        max_Scores[counter] = r2
                        max_ScoreDepths[counter] = depth
                    
                    TreeModelNames[counter] = model_name
                    counter = counter + 1
                print("Scores at Depth ", depth, " are ", max_Scores)
    
            for counter in [0,1,2]:
                print("For Model ", TreeModelNames[counter] ," and at Depth ", max_ScoreDepths[counter] , "Max score of ", max_Scores[counter], " is obtained")
            '''
            
            print("____________________________________________________________________________________________________")

            
            
        print("#######################################################################################################")
        print("#######################################################################################################")

#sys.stdout = old_stdout

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;New Test Size;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
Test Size Used is :  0.33
#######################################################################################################
#######################################################################################################


Scaler Used is  MinMaxScaler()
____________________________________________________________________________________________________
Models Trained With Scores are as follows:

[LinearRegression(), 'L

c:\users\tkarthik\project\maliciouspdf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-8-93954461c66b>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fit=model.fit(X_trainScaled, y_trainScaled)


Models Trained With Scores are as follows:

[LinearRegression(), 'LinearRegression', 0.002925956891930759]
[DecisionTreeRegressor(), 'DecisionTreeRegressor', 0.8647994115452792]
[GradientBoostingRegressor(n_estimators=200, random_state=0), 'GradientBoostingRegressor', 0.7061286919623528]
[RandomForestRegressor(n_estimators=200, random_state=0), 'RandomForestRegressor', 0.8254414925974765]


Default Decision Tree Depth with Scaler and setting  MinMaxScaler()  is  45


Default GradientBoostingRegressor Tree Depth with Scaler and setting  MinMaxScaler()  is  3


Default RandomForestRegressor Tree Depth with Scaler and setting  MinMaxScaler()  is  48
____________________________________________________________________________________________________
#######################################################################################################
#######################################################################################################
####################################

c:\users\tkarthik\project\maliciouspdf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-8-93954461c66b>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fit=model.fit(X_trainScaled, y_trainScaled)


Models Trained With Scores are as follows:

[LinearRegression(), 'LinearRegression', 0.0026893727055660177]
[DecisionTreeRegressor(), 'DecisionTreeRegressor', 0.7365857964931601]
[GradientBoostingRegressor(n_estimators=200, random_state=0), 'GradientBoostingRegressor', 0.7259533150504929]
[RandomForestRegressor(n_estimators=200, random_state=0), 'RandomForestRegressor', 0.8867560065211195]


Default Decision Tree Depth with Scaler and setting  StandardScaler()  is  44


Default GradientBoostingRegressor Tree Depth with Scaler and setting  StandardScaler()  is  3


Default RandomForestRegressor Tree Depth with Scaler and setting  StandardScaler()  is  51
____________________________________________________________________________________________________
#######################################################################################################
#######################################################################################################
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

...

Models Trained With Scores are as follows:

[LinearRegression(), 'LinearRegression', 0.0031390002076316925]
[DecisionTreeRegressor(), 'DecisionTreeRegressor', 0.9423963528957536]
[GradientBoostingRegressor(n_estimators=200, random_state=0), 'GradientBoostingRegressor', 0.8245451716197768]
[RandomForestRegressor(n_estimators=200, random_state=0), 'RandomForestRegressor', 0.9424065546602571]


Default Decision Tree Depth with Scaler and setting  StandardScaler()  is  49


Default GradientBoostingRegressor Tree Depth with Scaler and setting  StandardScaler()  is  3


Default RandomForestRegressor Tree Depth with Scaler and setting  StandardScaler()  is  49
____________________________________________________________________________________________________
#######################################################################################################
#######################################################################################################
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

c:\users\tkarthik\project\maliciouspdf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-8-93954461c66b>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fit=model.fit(X_trainScaled, y_trainScaled)


Models Trained With Scores are as follows:

[LinearRegression(), 'LinearRegression', 0.003347941046723646]
[DecisionTreeRegressor(), 'DecisionTreeRegressor', 0.9164516425870938]
[GradientBoostingRegressor(n_estimators=200, random_state=0), 'GradientBoostingRegressor', 0.7414073749412167]
[RandomForestRegressor(n_estimators=200, random_state=0), 'RandomForestRegressor', 0.9077614570419079]


Default Decision Tree Depth with Scaler and setting  MinMaxScaler()  is  44


Default GradientBoostingRegressor Tree Depth with Scaler and setting  MinMaxScaler()  is  3


Default RandomForestRegressor Tree Depth with Scaler and setting  MinMaxScaler()  is  49
____________________________________________________________________________________________________
#######################################################################################################
#######################################################################################################
####################################

c:\users\tkarthik\project\maliciouspdf\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-8-93954461c66b>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  fit=model.fit(X_trainScaled, y_trainScaled)


Models Trained With Scores are as follows:

[LinearRegression(), 'LinearRegression', 0.0008062904720144148]
[DecisionTreeRegressor(), 'DecisionTreeRegressor', 0.6091100210646634]
[GradientBoostingRegressor(n_estimators=200, random_state=0), 'GradientBoostingRegressor', 0.6563115231381023]
[RandomForestRegressor(n_estimators=200, random_state=0), 'RandomForestRegressor', 0.6976692781078938]


Default Decision Tree Depth with Scaler and setting  StandardScaler()  is  46


Default GradientBoostingRegressor Tree Depth with Scaler and setting  StandardScaler()  is  3


Default RandomForestRegressor Tree Depth with Scaler and setting  StandardScaler()  is  51
____________________________________________________________________________________________________
#######################################################################################################
#######################################################################################################
